## Laptime2Rank-evaluate-statusmodel

based on: Laptime2Rank-evaluate

go beyond curtrack and zerotrack by modeling the track status

rank prediction by laptime forecasting models

support:
+ train/test split by ratio or event
+ incremental training evaluation(adjust ratio)


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [2]:
import os
random.seed()
os.getcwd()
#GPUID = 1

'/scratch/hpda/indycar/notebook/12.StatusModel'

In [3]:
def load_data(event, year):
    inputfile = '../data/final/C_'+ event +'-' + year + '-final.csv'
    outputprefix = year +'-' + event + '-'
    dataset = pd.read_csv(inputfile)
    #dataset.info(verbose=True)    
    
    final_lap = max(dataset.completed_laps)
    total_laps = final_lap + 1

    # get records for the cars that finish the race
    completed_car_numbers= dataset[dataset.completed_laps == final_lap].car_number.values
    completed_car_count = len(completed_car_numbers)

    print('count of completed cars:', completed_car_count)
    print('completed cars:', completed_car_numbers)

    #make a copy
    alldata = dataset.copy()
    dataset = dataset[dataset['car_number'].isin(completed_car_numbers)]
    rankdata = alldata.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
    
    cldata = make_cl_data(dataset)
    acldata = make_cl_data(alldata)

    return alldata, rankdata, acldata

# make indy car completed_laps dataset
# car_number, completed_laps, rank, elapsed_time, rank_diff, elapsed_time_diff 
def make_cl_data(dataset):

    # pick up data with valid rank
    rankdata = dataset.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')

    # resort by car_number, lap
    uni_ds = rankdata.sort_values(by=['car_number', 'completed_laps', 'elapsed_time'], ascending=True)    
    #uni_ds = uni_ds.drop(["unique_id", "best_lap", "current_status", "track_status", "lap_status",
    #                  "laps_behind_leade","laps_behind_prec","overall_rank","pit_stop_count",
    #                  "last_pitted_lap","start_position","laps_led"], axis=1)
    
    uni_ds = uni_ds.drop(["unique_id", "best_lap", 
                      "laps_behind_leade","laps_behind_prec","overall_rank","pit_stop_count",
                      "last_pitted_lap","start_position","laps_led"], axis=1)
        
    carnumber = set(uni_ds['car_number'])
    print('cars:', carnumber)
    print('#cars=', len(carnumber))
   
    # faster solution , uni_ds already sorted by car_number and lap
    uni_ds['rank_diff'] = uni_ds['rank'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['rank_diff'][mask] = 0
    
    uni_ds['time_diff'] = uni_ds['elapsed_time'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['time_diff'][mask] = 0
    
    #df = uni_ds[['car_number','completed_laps','rank','elapsed_time','rank_diff','time_diff']]
    df = uni_ds[['car_number','completed_laps','rank','elapsed_time',
                 'rank_diff','time_diff',"current_status", "track_status", "lap_status"]]
    
    return df

In [20]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

def test_flag(a, bitflag):
    return (a & bitflag) ==  bitflag

#
# remove NaN at the tail
# there should be no nans in the middle of the ts
COL_LAPTIME=0
COL_RANK=1
COL_TRACKSTATUS=2
COL_LAPSTATUS=3
COL_TIMEDIFF=4
COL_CAUTION_LAPS_INSTINT=5
COL_LAPS_INSTINT= 6
MODE_ORACLE = 0
MODE_NOLAP = 1
MODE_NOTRACK = 2
MODE_TESTZERO = 4
MODE_TESTCURTRACK = 8

MODE_PREDTRACK = 16
MODE_PREDPIT = 32
#MODE_STR={MODE_ORACLE:'oracle', MODE_NOLAP:'nolap',MODE_NOTRACK:'notrack',MODE_TEST:'test'}

def make_dataset(runs, prediction_length, freq, 
                       useeid = False,
                       run_ts=COL_LAPTIME, 
                       train_ratio = 0.8,
                       use_global_dict = True,
                       oracle_mode = MODE_ORACLE,
                       test_cars = [],
                       half_moving_win = True 
                ):
    """
    split the ts to train and test part by the ratio
    
    oracle_mode: false to simulate prediction in real by 
        set the covariates of track and lap status as nan in the testset
            
    
    """    
    start = pd.Timestamp("01-01-2019", freq=freq)  # can be different for each time series

    train_set = []
    test_set = []
    
    #select run
    if runs>=0:
        _laptime_data = [laptime_data[runs].copy()]
    else:
        _laptime_data = laptime_data.copy()
    
   
    #_data: eventid, carids, datalist[carnumbers, features, lapnumber]->[laptime, rank, track, lap]]
    for _data in _laptime_data:
        _train = []
        _test = []
        
        #statistics on the ts length
        ts_len = [ _entry.shape[1] for _entry in _data[2]]
        max_len = int(np.max(ts_len))
        train_len = int(np.max(ts_len) * train_ratio)
        
        print(f'====event:{events[_data[0]]}, train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)}')
                
        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()
            
            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            rec = rec[:, ~np.isnan(rec[run_ts,:])]
            
            # remove short ts
            totallen = rec.shape[1]
            if ( totallen < train_len + prediction_length):
                print(f'a short ts: carid={_data[1][rowid]}，len={totallen}')
                continue                
            
            if use_global_dict:
                carno = _data[1][rowid]
                carid = global_carids[_data[1][rowid]]
            else:
                #simulation dataset, todo, fix the carids as decoder
                carno = rowid
                carid = rowid

            #eval on carids
            if test_cars and (carno not in test_cars):
                continue                
            
            if useeid:
                static_cat = [carid, _data[0]]    
            else:
                static_cat = [carid]    
                
            # selection of features
            if test_flag(oracle_mode, MODE_NOTRACK):                
                rec[COL_TRACKSTATUS, :] = 0
            if test_flag(oracle_mode, MODE_NOLAP):                
                rec[COL_LAPSTATUS, :] = 0
                
            # split and add to dataset record
            _train.append({'target': rec[run_ts,:train_len].astype(np.float32), 
                            'start': start, 
                            'feat_static_cat': static_cat,
                            'feat_dynamic_real': [rec[COL_TRACKSTATUS,:train_len],
                                   rec[COL_LAPSTATUS,:train_len]]
                          }
                          )
            
            # multiple test ts(rolling window as half of the prediction_length)
            test_rec_cnt = 0
            step = -int(prediction_length/2) if half_moving_win else -prediction_length
            for endpos in range(max_len, train_len+prediction_length, step):
                
                
                #check if enough for this ts
                if endpos > totallen:
                    continue
                        
                track_rec = rec[COL_TRACKSTATUS, :endpos].copy()
                lap_rec = rec[COL_LAPSTATUS, :endpos].copy()
                
                # test mode
                if test_flag(oracle_mode, MODE_TESTCURTRACK):
                    # since nan does not work, use cur-val instead
                    track_rec[-prediction_length:] = track_rec[-prediction_length - 1]
                    #track_rec[-prediction_length:] = random.randint(0,1)
                    #lap_rec[-prediction_length:] = lap_rec[-prediction_length - 1]
                    lap_rec[-prediction_length:] = 0
                elif test_flag(oracle_mode, MODE_TESTZERO):
                    #set prediction part as nan
                    #track_rec[-prediction_length:] = np.nan
                    #lap_rec[-prediction_length:] = np.nan
                    track_rec[-prediction_length:] = 0
                    lap_rec[-prediction_length:] = 0                    
                
                _test.append({'target': rec[run_ts,:endpos].astype(np.float32), 
                            'start': start, 
                            'feat_static_cat': static_cat,
                            'feat_dynamic_real': [track_rec,lap_rec]
                            #'feat_dynamic_real': [rec[COL_TRACKSTATUS,:endpos],
                            #       rec[COL_LAPSTATUS,:endpos]] 
                             }
                          )   
                test_rec_cnt += 1
            
            #add one ts
            print(f'carno:{carno}, totallen:{totallen}, nancount:{nan_count}, test_reccnt:{test_rec_cnt}')

        train_set.extend(_train)
        test_set.extend(_test)

    print(f'prediction_length:{prediction_length},train len:{len(train_set)}, test len:{len(test_set)}')
    
    train_ds = ListDataset(train_set, freq=freq)
    test_ds = ListDataset(test_set, freq=freq)    
    
    return train_ds, test_ds, train_set, test_set



# endpos -> vector of prediction_length
_track_pred  = {}
_track_true  = {}
def init_track_model():
    global _track_pred,_track_true
    _track_pred = {}
    _track_true  = {}
    
def get_track_model(track_rec, endpos, prediction_length, context_len=10):
    """
    return the predicted track status
    """
    global _track_pred,_track_true
    # this is the perfect track model for Indy500 2018
    track_model = [6,4,4,5,6,6,4]
    if endpos in _track_pred:
        return _track_pred[endpos]
    else:
        #get yflag lap count from the start pred point
        yflaplen = 0
        for i in range(1, context_len):
            if track_rec[- prediction_length - i] == 1:
                yflaplen += 1
            else:
                break
                
        #laps remain, fill into the future
        trackpred = np.array([0 for x in range(prediction_length)])
        
        yflap_pred = random.choice(track_model)
        if yflaplen > 0 and yflap_pred > yflaplen:
            trackpred[:(yflap_pred - yflaplen)] = 1
        _track_pred[endpos] = trackpred
        
        _track_true[endpos]  = track_rec[- prediction_length:].copy()
        
        return trackpred

# pit model is separate for each car
def get_pit_model(cuation_laps_instint, laps_instint, prediction_length):
    """
    return the predicted pit status
    """
    # this is the perfect empirical pit model for Indy500 2018
    pit_model_all = [[33, 32, 35, 32, 35, 34, 35, 34, 37, 32, 37, 30, 33, 36, 35, 33, 36, 30, 31, 33, 36, 37, 35, 34, 34, 33, 37, 35, 39, 32, 36, 35, 34, 32, 36, 32, 31, 36, 33, 33, 35, 37, 40, 32, 32, 34, 35, 36, 33, 37, 35, 37, 34, 35, 39, 32, 31, 37, 32, 35, 36, 39, 35, 36, 34, 35, 33, 33, 34, 32, 33, 34],
                [45, 44, 46, 44, 43, 46, 45, 43, 41, 48, 46, 43, 47, 45, 49, 44, 48, 42, 44, 46, 45, 45, 43, 44, 44, 43, 46]]
    pit_model_top8 = [[33, 32, 35, 33, 36, 33, 36, 33, 37, 35, 36, 33, 37, 34],
                 [46, 45, 43, 48, 46, 45, 45, 43]]
    
    pit_model = pit_model_all
    
    if cuation_laps_instint>10:
        #use low model
        pred_pit_laps = random.choice(pit_model[0])
    else:
        pred_pit_laps = random.choice(pit_model[1])
                
    #laps remain, fill into the future
    pitpred = np.array([0 for x in range(prediction_length)])
    
    if (pred_pit_laps > laps_instint) and (pred_pit_laps <= laps_instint + prediction_length):
        pitpred[pred_pit_laps - laps_instint - 1] = 1
         
    return pitpred    
    
def make_dataset_byevent(runs, prediction_length, freq, 
                       useeid = False,
                       run_ts=COL_LAPTIME, 
                       test_event = 'Indy500',
                       test_cars = [],  
                       use_global_dict = True,
                       oracle_mode = MODE_ORACLE,
                       half_moving_win = True,
                       train_ratio=0.8
                ):
    """
    split the ts to train and test part by the ratio
    
    oracle_mode: false to simulate prediction in real by 
        set the covariates of track and lap status as nan in the testset
            
    
    """    
    init_track_model()
    
    start = pd.Timestamp("01-01-2019", freq=freq)  # can be different for each time series

    train_set = []
    test_set = []
    
    #select run
    if runs>=0:
        _laptime_data = [laptime_data[runs].copy()]
    else:
        _laptime_data = laptime_data.copy()
    
   
    #_data: eventid, carids, datalist[carnumbers, features, lapnumber]->[laptime, rank, track, lap]]
    for _data in _laptime_data:
        _train = []
        _test = []
        
        if events[_data[0]] == test_event:
            test_mode = True
        
        else:
            test_mode = False
            
            
        #statistics on the ts length
        ts_len = [ _entry.shape[1] for _entry in _data[2]]
        max_len = int(np.max(ts_len))
        train_len = int(np.max(ts_len) * train_ratio)
        
        
        print(f'====event:{events[_data[0]]}, train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)}')
                
        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()
            
            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            rec = rec[:, ~np.isnan(rec[run_ts,:])]
            
            # remove short ts
            totallen = rec.shape[1]
            if ( totallen < train_len + prediction_length):
                print(f'a short ts: carid={_data[1][rowid]}，len={totallen}')
                continue                
            
            if use_global_dict:
                carno = _data[1][rowid]
                carid = global_carids[_data[1][rowid]]
            else:
                #simulation dataset, todo, fix the carids as decoder
                carno = rowid
                carid = rowid
                
            
            if useeid:
                static_cat = [carid, _data[0]]    
            else:
                static_cat = [carid]    
                
            # selection of features
            if test_flag(oracle_mode, MODE_NOTRACK):                
                rec[COL_TRACKSTATUS, :] = 0
            if test_flag(oracle_mode, MODE_NOLAP):                
                rec[COL_LAPSTATUS, :] = 0

            test_rec_cnt = 0
            if not test_mode:
                
                # all go to train set
                _train.append({'target': rec[run_ts,:].astype(np.float32), 
                                'start': start, 
                                'feat_static_cat': static_cat,
                                'feat_dynamic_real': [rec[COL_TRACKSTATUS,:],
                                       rec[COL_LAPSTATUS,:]]
                              }
                              )
            else:
                # reset train_len
                #context_len = prediction_length*2
                #if context_len < 10:
                #    context_len = 10
                
                context_len = train_len
                
                # multiple test ts(rolling window as half of the prediction_length)

                step = -int(prediction_length/2) if half_moving_win else -prediction_length
                
                #bug fix, fixed the split point for all cars/ts
                for endpos in range(max_len, context_len+prediction_length, 
                                    step):

                    #check if enough for this ts
                    if endpos > totallen:
                        continue
                    
                    track_rec = rec[COL_TRACKSTATUS, :endpos].copy()
                    lap_rec = rec[COL_LAPSTATUS, :endpos].copy()
                    
                    caution_laps_instint = int(rec[COL_CAUTION_LAPS_INSTINT, endpos -prediction_length - 1])
                    laps_instint = int(rec[COL_LAPS_INSTINT, endpos -prediction_length - 1])
                    

                    # test mode
                    if test_flag(oracle_mode, MODE_TESTCURTRACK):
                        # since nan does not work, use cur-val instead
                        track_rec[-prediction_length:] = track_rec[-prediction_length - 1]
                        #track_rec[-prediction_length:] = random.randint(0,1)
                        #lap_rec[-prediction_length:] = lap_rec[-prediction_length - 1]
                        lap_rec[-prediction_length:] = 0
                    elif test_flag(oracle_mode, MODE_PREDTRACK):
                        predrec = get_track_model(track_rec, endpos, prediction_length)
                        track_rec[-prediction_length:] = predrec
                        lap_rec[-prediction_length:] = 0
                    elif test_flag(oracle_mode, MODE_PREDPIT):
                        predrec = get_track_model(track_rec, endpos, prediction_length)
                        track_rec[-prediction_length:] = predrec
                        lap_rec[-prediction_length:] = get_pit_model(caution_laps_instint,
                                                                    laps_instint,prediction_length)
                    elif test_flag(oracle_mode, MODE_TESTZERO):
                        #set prediction part as nan
                        #track_rec[-prediction_length:] = np.nan
                        #lap_rec[-prediction_length:] = np.nan
                        track_rec[-prediction_length:] = 0
                        lap_rec[-prediction_length:] = 0                    

                    _test.append({'target': rec[run_ts,:endpos].astype(np.float32), 
                                'start': start, 
                                'feat_static_cat': static_cat,
                                'feat_dynamic_real': [track_rec,lap_rec]
                                #'feat_dynamic_real': [rec[COL_TRACKSTATUS,:endpos],
                                #       rec[COL_LAPSTATUS,:endpos]] 
                                 }
                              )   
                    test_rec_cnt += 1
            
            #add one ts
            print(f'carno:{carno}, totallen:{totallen}, nancount:{nan_count}, test_reccnt:{test_rec_cnt}')

        train_set.extend(_train)
        test_set.extend(_test)

    print(f'train len:{len(train_set)}, test len:{len(test_set)}')
    
    train_ds = ListDataset(train_set, freq=freq)
    test_ds = ListDataset(test_set, freq=freq)    
    
    return train_ds, test_ds, train_set, test_set

def save_dataset(datafile,freq, prediction_length, cardinality, train_ds, test_ds):
    with open(datafile, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = [freq, prediction_length, cardinality, train_ds, test_ds]
        #savedata = [freq, train_set, test_set]
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)        

### test for Indy500

In [5]:
def predict(test_ds,predictor):
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=test_ds,  # test dataset
        predictor=predictor,  # predictor
        num_samples=100,  # number of sample paths we want for evaluation
    )

    forecasts = list(forecast_it)
    tss = list(ts_it)
    print(f'tss len={len(tss)}, forecasts len={len(forecasts)}')
    
    return tss, forecasts

   
def run_prediction(test_ds, prediction_length, trainid):
    with mx.Context(mx.gpu(7)):    
        pred_ret = {}

        rootdir = f'../models/remote/laptime-{trainid}/'
        # deepAR-Oracle
        model_name = 'deepAR-Oracle-curtrack'
        model=f'deepAR-Oracle-laptime-curtrack-indy-f1min-t{prediction_length}-e1000-r1_curtrack_t{prediction_length}'
        modeldir = rootdir + model
        print(f'predicting model={model_name}, plen={prediction_length}')
        predictor =  Predictor.deserialize(Path(modeldir))
        print(f'loading model...done!, ctx:{predictor.ctx}')
        tss, forecasts = predict(test_ds,predictor)
        pred_ret[model_name] = [tss, forecasts]

        # deepAR-Oracle
        model_name = 'deepAR-Oracle'
        model=f'deepAR-Oracle-laptime-all-indy-f1min-t{prediction_length}-e1000-r1_oracle_t{prediction_length}'
        modeldir = rootdir + model
        print(f'predicting model={model_name}, plen={prediction_length}')
        predictor =  Predictor.deserialize(Path(modeldir))
        print(f'loading model...done!, ctx:{predictor.ctx}')
        tss, forecasts = predict(test_ds,predictor)
        pred_ret[model_name] = [tss, forecasts]

        # deepAR
        model_name = 'deepAR'
        model=f'deepAR-laptime-all-indy-f1min-t{prediction_length}-e1000-r1_deepar_t{prediction_length}'
        modeldir = rootdir + model
        print(f'predicting model={model_name}, plen={prediction_length}')
        predictor =  Predictor.deserialize(Path(modeldir))
        print(f'loading model...done!, ctx:{predictor.ctx}')
        tss, forecasts = predict(test_ds,predictor)
        pred_ret[model_name] = [tss, forecasts]

        # naive
        model_name = 'naive'
        print(f'predicting model={model_name}, plen={prediction_length}')
        predictor =  NaivePredictor(freq= freq, prediction_length = prediction_length)
        tss, forecasts = predict(test_ds,predictor)
        pred_ret[model_name] = [tss, forecasts]

        # arima
        model_name = 'arima'
        print(f'predicting model={model_name}, plen={prediction_length}')
        predictor =  RForecastPredictor(method_name='arima',freq= freq, 
                                        prediction_length = prediction_length,trunc_length=60)
        #tss, forecasts = predict(test_ds,predictor)
        pred_ret[model_name] = [tss, forecasts]

        return pred_ret    
    
def run_prediction_ex(test_ds, prediction_length, model_name,trainid):
    with mx.Context(mx.gpu(7)):    
        pred_ret = []

        rootdir = f'../models/remote/laptime-{trainid}/'
        # deepAR-Oracle
        if model_name == 'curtrack':
            model=f'deepAR-Oracle-laptime-curtrack-indy-f1min-t{prediction_length}-e1000-r1_curtrack_t{prediction_length}'
            modeldir = rootdir + model
            print(f'predicting model={model_name}, plen={prediction_length}')
            predictor =  Predictor.deserialize(Path(modeldir))
            print(f'loading model...done!, ctx:{predictor.ctx}')
            tss, forecasts = predict(test_ds,predictor)
            pred_ret = [tss, forecasts]

        elif model_name == 'zerotrack':
            model=f'deepAR-Oracle-laptime-nolap-zerotrack-indy-f1min-t{prediction_length}-e1000-r1_zerotrack_t{prediction_length}'
            modeldir = rootdir + model
            print(f'predicting model={model_name}, plen={prediction_length}')
            predictor =  Predictor.deserialize(Path(modeldir))
            print(f'loading model...done!, ctx:{predictor.ctx}')
            tss, forecasts = predict(test_ds,predictor)
            pred_ret = [tss, forecasts]
            
        # deepAR-Oracle
        elif model_name == 'oracle':
            model=f'deepAR-Oracle-laptime-all-indy-f1min-t{prediction_length}-e1000-r1_oracle_t{prediction_length}'
            modeldir = rootdir + model
            print(f'predicting model={model_name}, plen={prediction_length}')
            predictor =  Predictor.deserialize(Path(modeldir))
            print(f'loading model...done!, ctx:{predictor.ctx}')
            tss, forecasts = predict(test_ds,predictor)
            pred_ret = [tss, forecasts]

        # deepAR
        elif model_name == 'deepAR':
            model=f'deepAR-laptime-all-indy-f1min-t{prediction_length}-e1000-r1_deepar_t{prediction_length}'
            modeldir = rootdir + model
            print(f'predicting model={model_name}, plen={prediction_length}')
            predictor =  Predictor.deserialize(Path(modeldir))
            print(f'loading model...done!, ctx:{predictor.ctx}')
            tss, forecasts = predict(test_ds,predictor)
            pred_ret = [tss, forecasts]

        # naive
        elif model_name == 'naive':
            print(f'predicting model={model_name}, plen={prediction_length}')
            predictor =  NaivePredictor(freq= freq, prediction_length = prediction_length)
            tss, forecasts = predict(test_ds,predictor)
            pred_ret = [tss, forecasts]

        # arima
        elif model_name == 'arima':
            print(f'predicting model={model_name}, plen={prediction_length}')
            predictor =  RForecastPredictor(method_name='arima',freq= freq, 
                                            prediction_length = prediction_length,trunc_length=60)
            tss, forecasts = predict(test_ds,predictor)
            pred_ret = [tss, forecasts]
        else:
            print(f'error: model {model_name} not support yet!')

        return pred_ret     

In [6]:
#calc rank
def eval_rank_bytimediff(test_ds,tss,forecasts,prediction_length):
    """
    timediff models
    
    works for one event only
    
    """

    carlist = []

    # carno-lap# -> elapsed_time[] array
    forecasts_et = dict()

    ds_iter =  iter(test_ds)
    for idx in range(len(test_ds)):
        test_rec = next(ds_iter)
        #global carid
        carno = decode_carids[test_rec['feat_static_cat'][0]]
        #print('car no:', carno)

        if carno not in carlist:
            carlist.append(carno)

        # calc elapsed time
        prediction_len = forecasts[idx].samples.shape[1]
        if prediction_length != prediction_len:
            print('error: prediction_len does not match, {prediction_length}:{prediction_len}')
            return []
        
        #forecast_laptime_mean = np.mean(forecasts[idx].samples, axis=0).reshape((prediction_len,1))
        forecast_laptime_mean = np.median(forecasts[idx].samples, axis=0).reshape((prediction_len,1))
        
        timediff_array = tss[idx].values.copy()

        #save the prediction
        completed_laps = len(tss[idx]) - prediction_len + 1
        #print('car no:', carno, 'completed_laps:', completed_laps)
        #key = '%s-%s'%(carno, completed_laps)
        #forecasts_et[key] = elapsed_time[-prediction_len:].copy()
        if completed_laps not in forecasts_et:
            forecasts_et[completed_laps] = {}
        forecasts_et[completed_laps][carno] = [timediff_array[-prediction_len:].copy(),
                                                   forecast_laptime_mean.copy()]


    # calc rank
    rank_ret = []
    for lap in forecasts_et.keys():
        #get car list for this lap
        carlist = list(forecasts_et[lap].keys())
        #print('carlist:', carlist)

        caridmap={key:idx for idx, key in enumerate(carlist)}

        #fill in data
        time_diff = np.zeros((2, len(carlist), prediction_len))
        for carno in carlist:
            carid = caridmap[carno]
            time_diff[0, carid, :] = forecasts_et[lap][carno][0].reshape((prediction_len))
            time_diff[1, carid, :] = forecasts_et[lap][carno][1].reshape((prediction_len))

        #calculate rank    
        idx = np.argsort(time_diff[0], axis=0)
        true_rank = np.argsort(idx, axis=0)

        idx = np.argsort(time_diff[1], axis=0)
        pred_rank = np.argsort(idx, axis=0)

        rank_ret.append([lap, time_diff, true_rank, pred_rank])
        
    return rank_ret,forecasts_et
    
#calc rank
def eval_rank(test_ds,tss,forecasts,prediction_length):
    """
    dependency:
        start_offset[]; for one specific event
    """

    carlist = []

    # carno-lap# -> elapsed_time[] array
    forecasts_et = dict()

    ds_iter =  iter(test_ds)
    for idx in range(len(test_ds)):
        test_rec = next(ds_iter)
        #global carid
        carno = decode_carids[test_rec['feat_static_cat'][0]]
        #print('car no:', carno)

        if carno not in carlist:
            carlist.append(carno)

        #start_offset is global var
        offset = start_offset[(start_offset['car_number']==carno)].elapsed_time.values[0] 
        #print('start_offset:', offset)

        # calc elapsed time
        prediction_len = forecasts[idx].samples.shape[1]
        if prediction_length != prediction_len:
            print('error: prediction_len does not match, {prediction_length}:{prediction_len}')
            return []
        
        forecast_laptime_mean = np.mean(forecasts[idx].samples, axis=0).reshape((prediction_len,1))
        #forecast_laptime_mean = np.median(forecasts[idx].samples, axis=0).reshape((prediction_len,1))

        laptime_array = tss[idx].values.copy()
        elapsed_time = np.cumsum(laptime_array) + offset

        laptime_array = tss[idx].values.copy()
        laptime_array[-prediction_len:] = forecast_laptime_mean 
        elapsed_time_hat = np.cumsum(laptime_array) + offset

        #save the prediction
        completed_laps = len(tss[idx]) - prediction_len + 1
        #print('car no:', carno, 'completed_laps:', completed_laps)
        #key = '%s-%s'%(carno, completed_laps)
        #forecasts_et[key] = elapsed_time[-prediction_len:].copy()
        if completed_laps not in forecasts_et:
            forecasts_et[completed_laps] = {}
        #forecasts_et[completed_laps][carno] = [elapsed_time[-prediction_len-1:].copy(),
        #                                           elapsed_time_hat[-prediction_len-1:].copy()]
        forecasts_et[completed_laps][carno] = [elapsed_time[-prediction_len:].copy(),
                                                   elapsed_time_hat[-prediction_len:].copy()]


    # calc rank
    rank_ret = []
    for lap in forecasts_et.keys():
        #get car list for this lap
        carlist = list(forecasts_et[lap].keys())
        #print('carlist:', carlist)

        caridmap={key:idx for idx, key in enumerate(carlist)}

        #fill in data
        #elapsed_time = np.zeros((2, len(carlist), prediction_len+1))
        elapsed_time = np.zeros((2, len(carlist), prediction_len))
        for carno in carlist:
            carid = caridmap[carno]
            elapsed_time[0, carid, :] = forecasts_et[lap][carno][0]
            elapsed_time[1, carid, :] = forecasts_et[lap][carno][1]

        #calculate rank    
        idx = np.argsort(elapsed_time[0], axis=0)
        true_rank = np.argsort(idx, axis=0)

        idx = np.argsort(elapsed_time[1], axis=0)
        pred_rank = np.argsort(idx, axis=0)

        rank_ret.append([lap, elapsed_time, true_rank, pred_rank])
        
    return rank_ret,forecasts_et
   
def get_acc(rank_ret,prediction_length):    
    # evaluate
    #top1 accuracy
    top1acc = 0
    top1acc_farmost = 0
    top5acc = 0
    top5acc_farmost = 0
    tau = 0
    rmse = 0.
    
    for rec in rank_ret:
        trueRank = rec[2]
        predRank = rec[3]

        #top1 , rank = 0, first col is not prediction
        top1acc += np.sum((trueRank==0) & (predRank==0)) 
        
        top1acc_farmost += np.sum((trueRank[:,-1]==0) & (predRank[:,-1]==0))
        
        #top5
        top5acc += np.sum((trueRank<5) & (predRank<5)) 
        
        top5acc_farmost += np.sum((trueRank[:,-1]<5) & (predRank[:,-1]<5))
        
        # tau
        tao, _ = stats.kendalltau(trueRank, predRank)
        tau += tao
        
        #rmse
        rmse += mean_squared_error(predRank,trueRank)
        

    top1acc = top1acc *1.0/ (len(rank_ret)*prediction_length)
    top1acc_farmost = top1acc_farmost *1.0/ (len(rank_ret))
    top5acc = top5acc *1.0/ (5*len(rank_ret)*prediction_length)
    top5acc_farmost = top5acc_farmost *1.0/ (5*len(rank_ret))
    tau = tau/len(rank_ret)
    rmse = rmse/len(rank_ret)
    
        
    print(f'total:{len(rank_ret)}, prediction_length:{prediction_length}') 
    print('top1acc=', top1acc,
          'top1acc_farmost=', top1acc_farmost,
          'top5acc=', top5acc,
          'top5acc_farmost=', top5acc_farmost,
         )
    print('tau = ', tau,
         'rmse = ', rmse)
    
    return (top1acc,top1acc_farmost,top5acc,top5acc_farmost,tau,rmse)
    
def get_top1acc_farmost(rank_ret,prediction_len):    
    # evaluate
    #top1 accuracy
    hitcnt = 0
    for rec in rank_ret:
        trueRank = rec[2]
        predRank = rec[3]

        #top1 , rank = 0, first col is not prediction
        hitcnt += np.sum((trueRank[:,-1]==0) & (predRank[:,-1]==0)) 

    print('total:', hitcnt, 'top1acc_farmost=', hitcnt *1.0/ (len(rank_ret)*prediction_length))    



In [7]:
def run_exp_old(prediction_length, half_moving_win, train_ratio=0.8):
    ### create test dataset
    test_cars = []
    train_ds, test_ds,_,_ = make_dataset_byevent(events_id[test_event], prediction_length,freq, 
                                         oracle_mode=MODE_TESTCURTRACK,
                                         run_ts = COL_LAPTIME,
                                         test_cars=test_cars,
                                         half_moving_win= half_moving_win,
                                         train_ratio=train_ratio)
    pred_ret = run_prediction(test_ds, prediction_length)

    models = ['deepAR-Oracle','deepAR-Oracle-curtrack','deepAR','naive','arima']
    retdf = []
    for model in models:
        print('model:', model)
        tss, forecasts = pred_ret[model]

        rank_ret,_ = eval_rank(test_ds,tss,forecasts,prediction_length)
        metrics = get_acc(rank_ret,prediction_length)
        ret = [model, prediction_length, half_moving_win]
        ret.extend(metrics)
        retdf.append(ret)
        
    return pred_ret, test_ds, retdf

def run_exp(prediction_length, half_moving_win, train_ratio=0.8, trainid="r0.8"):
    """
    dependency: test_event, test on one event only
    
    """
    
    test_cars = []
    models = ['oracle','deepAR','naive']
    #,'arima']
    retdf = []
    pred_ret = {}
    ds_ret = {}
    
    ### create test dataset
    train_ds, test_ds,_,_ = make_dataset_byevent(events_id[test_event], prediction_length,freq, 
                                         oracle_mode=MODE_ORACLE,
                                         run_ts = COL_LAPTIME,
                                         test_cars=test_cars,
                                         half_moving_win= half_moving_win,
                                         train_ratio=train_ratio)
    
    for model in models:
        print('model:', model)
        tss, forecasts = run_prediction_ex(test_ds, prediction_length, model,trainid=trainid)
        pred_ret[model] = [tss, forecasts]
        ds_ret[model] = test_ds

        rank_ret,_ = eval_rank(test_ds,tss,forecasts,prediction_length)
        metrics = get_acc(rank_ret,prediction_length)
        ret = [model, prediction_length, half_moving_win,trainid]
        ret.extend(metrics)
        retdf.append(ret)
    
    # special model with test_ds
    models = ['curtrack']        
    train_ds, test_ds,_,_ = make_dataset_byevent(events_id[test_event], prediction_length,freq, 
                                         oracle_mode=MODE_TESTCURTRACK,
                                         run_ts = COL_LAPTIME,
                                         test_cars=test_cars,
                                         half_moving_win= half_moving_win,
                                         train_ratio=train_ratio)
    
    for model in models:
        print('model:', model)
        tss, forecasts = run_prediction_ex(test_ds, prediction_length, model,trainid=trainid)
        pred_ret[model] = [tss, forecasts]
        ds_ret[model] = test_ds
        
        rank_ret,_ = eval_rank(test_ds,tss,forecasts,prediction_length)
        metrics = get_acc(rank_ret,prediction_length)
        ret = [model, prediction_length, half_moving_win,trainid]
        ret.extend(metrics)
        retdf.append(ret)

    # zerotrack
    models = ['zerotrack']        
    train_ds, test_ds,_,_ = make_dataset_byevent(events_id[test_event], prediction_length,freq, 
                                         oracle_mode=MODE_TESTZERO,
                                         run_ts = COL_LAPTIME,
                                         test_cars=test_cars,
                                         half_moving_win= half_moving_win,
                                         train_ratio=train_ratio)
    
    for model in models:
        print('model:', model)
        tss, forecasts = run_prediction_ex(test_ds, prediction_length, model,trainid=trainid)
        pred_ret[model] = [tss, forecasts]
        ds_ret[model] = test_ds
        
        rank_ret,_ = eval_rank(test_ds,tss,forecasts,prediction_length)
        metrics = get_acc(rank_ret,prediction_length)
        ret = [model, prediction_length, half_moving_win,trainid]
        ret.extend(metrics)
        retdf.append(ret)
    
        
    return pred_ret, ds_ret, retdf



def run_exp_predtrack(prediction_length, half_moving_win, train_ratio=0.8, trainid="r0.8"):
    """
    dependency: test_event, test on one event only
    
    """
    
    test_cars = []
    models = ['oracle','curtrack','zerotrack']
    #,'arima']
    retdf = []
    pred_ret = {}
    ds_ret = {}
    
    ### create test dataset
    train_ds, test_ds,_,_ = make_dataset_byevent(events_id[test_event], prediction_length,freq, 
                                         oracle_mode=MODE_PREDTRACK,
                                         run_ts = COL_LAPTIME,
                                         test_cars=test_cars,
                                         half_moving_win= half_moving_win,
                                         train_ratio=train_ratio)
    
    for model in models:
        print('model:', model)
        tss, forecasts = run_prediction_ex(test_ds, prediction_length, model,trainid=trainid)
        pred_ret[model] = [tss, forecasts]
        ds_ret[model] = test_ds

        rank_ret,_ = eval_rank(test_ds,tss,forecasts,prediction_length)
        metrics = get_acc(rank_ret,prediction_length)
        ret = [model, prediction_length, half_moving_win,trainid]
        ret.extend(metrics)
        retdf.append(ret)
    
    return pred_ret, ds_ret, retdf

def run_exp_predpit(prediction_length, half_moving_win, train_ratio=0.8, trainid="r0.8"):
    """
    dependency: test_event, test on one event only
    
    """
    
    test_cars = []
    models = ['oracle','curtrack','zerotrack']
    #,'arima']
    retdf = []
    pred_ret = {}
    ds_ret = {}
    
    ### create test dataset
    train_ds, test_ds,_,_ = make_dataset_byevent(events_id[test_event], prediction_length,freq, 
                                         oracle_mode=MODE_PREDPIT,
                                         run_ts = COL_LAPTIME,
                                         test_cars=test_cars,
                                         half_moving_win= half_moving_win,
                                         train_ratio=train_ratio)
    
    for model in models:
        print('model:', model)
        tss, forecasts = run_prediction_ex(test_ds, prediction_length, model,trainid=trainid)
        pred_ret[model] = [tss, forecasts]
        ds_ret[model] = test_ds

        rank_ret,_ = eval_rank(test_ds,tss,forecasts,prediction_length)
        metrics = get_acc(rank_ret,prediction_length)
        ret = [model, prediction_length, half_moving_win,trainid]
        ret.extend(metrics)
        retdf.append(ret)
    
    return pred_ret, ds_ret, retdf

### init

In [8]:
#
# parameters
#
#year = '2017'
year = '2018'
#event = 'Toronto'
#https://www.racing-reference.info/season-stats/2018/O/#
events_totalmiles=[256,500,372,268,500,310]
events_laplen = [1.022,2.5,1.5,0.894,2.5,1.25]
events = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
#events = ['Gateway']

#events = ['Indy500']
#events = ['Phoenix']
events_id={key:idx for idx, key in enumerate(events)}
#works for only one event


In [9]:
stagedata = {}
global_carids = {}
traindata = None
cur_carid = 0
for event in events:
    #alldata, rankdata, acldata, flagdata
    stagedata[event] = load_data(event, year)
    
    alldata, rankdata, acldata = stagedata[event]
    carlist = set(acldata['car_number'])
    laplist = set(acldata['completed_laps'])
    print('%s: carno=%d, lapnum=%d'%(event, len(carlist), len(laplist)))

    #build the carid map
    for car in carlist:
        if car not in global_carids:
            global_carids[car] = cur_carid
            cur_carid += 1

count of completed cars: 11
completed cars: [ 1  6 27  9 28  5 20 14 15 22 30]
cars: {1, 5, 6, 9, 14, 15, 20, 22, 27, 28, 30}
#cars= 11
cars: {1, 4, 5, 6, 9, 10, 12, 14, 15, 18, 19, 20, 21, 22, 23, 26, 27, 28, 30, 32, 59, 88, 98}
#cars= 23
Phoenix: carno=23, lapnum=251
count of completed cars: 18
completed cars: [12 20  9 27 28 22 29  1  6 15 66 98  4 88 25 60 64 23]
cars: {64, 1, 66, 98, 4, 6, 9, 12, 60, 15, 20, 22, 23, 88, 25, 27, 28, 29}
#cars= 18
cars: {1, 3, 4, 6, 7, 9, 10, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 59, 60, 64, 66, 88, 98}
#cars= 33
Indy500: carno=33, lapnum=201


/scratch/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/scratch/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


count of completed cars: 9
completed cars: [ 9 22 27  5 28 15 30 18 10]
cars: {5, 9, 10, 15, 18, 22, 27, 28, 30}
#cars= 9
cars: {1, 3, 4, 5, 6, 7, 9, 10, 12, 14, 15, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 30, 47, 55, 57, 59, 60, 68, 73, 83, 88, 98}
#cars= 32
Texas: carno=32, lapnum=249
count of completed cars: 5
completed cars: [ 5 21 30  1  6]
cars: {1, 5, 6, 21, 30}
#cars= 5
cars: {1, 4, 5, 6, 9, 10, 12, 14, 15, 18, 19, 20, 21, 22, 23, 26, 27, 28, 30, 59, 88, 98}
#cars= 22
Iowa: carno=22, lapnum=301
count of completed cars: 4
completed cars: [27 12  9 18]
cars: {9, 18, 27, 12}
#cars= 4
cars: {1, 4, 5, 6, 9, 10, 12, 14, 15, 18, 19, 20, 21, 22, 23, 26, 27, 28, 30, 59, 88, 98}
#cars= 22
Pocono: carno=22, lapnum=201
count of completed cars: 8
completed cars: [12 27  9 22 26 21  1 10]
cars: {1, 9, 10, 12, 21, 22, 26, 27}
#cars= 8
cars: {1, 4, 5, 9, 10, 12, 14, 15, 18, 19, 20, 21, 22, 23, 26, 27, 28, 30, 59, 88, 98}
#cars= 21
Gateway: carno=21, lapnum=249


In [10]:
event='Indy500'
test_event = event
alldata, rankdata, acldata = stagedata[event]
final_lap = max(rankdata.completed_laps)
completed_car_numbers= rankdata[rankdata.completed_laps == final_lap].car_number.values

start_offset = rankdata[rankdata['completed_laps']==0][['car_number','elapsed_time']]

In [11]:
# start from here
import pickle
with open('laptime_rank_timediff_pit-oracle-%s.pickle'%year, 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    global_carids, laptime_data = pickle.load(f, encoding='latin1')

In [12]:
freq = "1min"
#decode global_carids
decode_carids={carid:carno for carno, carid in global_carids.items()}
    
#useeid = False
#interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
#ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
#if useeid:
#    cardinality = [len(global_carids), len(laptime_data)]
#else:
#    cardinality = [len(global_carids)]

In [ ]:
def 

### loop test

In [23]:
half=[True, False]
#plens=[2,5,10,20,30]
plens=[2,5,10]
half=[False]
trainids = ["indy500-r0.2","indy500-r0.4","indy500"]
#trainids = ["r0.5","r0.6"]
#half=[True,False]
#plens=[2]
runs = 5

#exp_id='median-r0.8'
exp_id='mean-splitbyevent-predpit'



allret = []
for runid in range(runs):
    exp_data = []
    exp_result = []
    
    for halfmode in half:
        for plen in plens:
            for trainid in trainids:
                print('='*30)
                pred_ret, test_ds, metric_ret = run_exp_predpit(plen, halfmode, 
                                                        train_ratio=0.4,
                                                        trainid=trainid)

                #save 
                exp_data.append((pred_ret, test_ds))
                exp_result.extend(metric_ret)

    #save result
    result = pd.DataFrame(exp_result, columns = ['model' , 'prediction_length', 'halfmode',
                                       'trainid',
                                       'top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse'])
    
    #result['runid'] = [runid for x in range(len(result))]
    allret.append(result)


#final
rowcnt = len(allret[0])
metrics = np.empty((runs, rowcnt, 6))
for runid, ret in enumerate(allret):
    metrics[runid, :,:] = ret[['top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse']].values
    

#average
average = np.mean(metrics[:,:,:], axis=0)
result = allret[0][['model' , 'prediction_length', 'halfmode',
                                       'trainid',
                               'top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse']].values
result[:,4:] = average

dfret = pd.DataFrame(result, columns = ['model' , 'prediction_length', 'halfmode',
                                   'trainid',
                                   'top1acc','top1acc_farmost','top5acc',
                                   'top5acc_farmost','tau','rmse'])

dfret.to_csv(f'laptime2rank-evaluate-indy500-{exp_id}-r{runs}-result.csv')

INFO:root:Using GPU


====event:Indy500, train_len=80, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:59
carno:3, totallen:146, nancount:54, test_reccnt:32
carno:4, totallen:200, nancount:0, test_reccnt:59
carno:6, totallen:200, nancount:0, test_reccnt:59
carno:7, totallen:193, nancount:7, test_reccnt:55
carno:9, totallen:200, nancount:0, test_reccnt:59
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:59
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:52
carno:15, totallen:200, nancount:0, test_reccnt:59
carno:17, totallen:199, nancount:1, test_reccnt:58
carno:18, totallen:137, nancount:63, test_reccnt:27
carno:19, totallen:199, nancount:1, test_reccnt:58
carno:20, totallen:200, nancount:0, test_reccnt:59
carno:21, totallen:199, nancount:1, test_reccnt:58
carno:22, totallen:200, nancount:0, test_reccnt:59
carno:23, totallen:200, nancount:0, test_reccnt:59
carno:24, totallen:154, nancount:46, test_reccnt:36
carno:25, totallen:20

INFO:root:Using GPU


total:59, prediction_length:2
top1acc= 0.6694915254237288 top1acc_farmost= 0.6101694915254238 top5acc= 0.7966101694915254 top5acc_farmost= 0.7254237288135593
tau =  0.8059299858392047 rmse =  14.307602929563137
model: curtrack
predicting model=curtrack, plen=2
loading model...done!, ctx:gpu(0)
tss len=1568, forecasts len=1568


INFO:root:Using GPU


...

INFO:root:Using GPU


total:11, prediction_length:10
top1acc= 0.32727272727272727 top1acc_farmost= 0.18181818181818182 top5acc= 0.5 top5acc_farmost= 0.3090909090909091
tau =  0.4781781635358758 rmse =  44.60908578012026
model: curtrack
predicting model=curtrack, plen=10
loading model...done!, ctx:gpu(0)
tss len=284, forecasts len=284


INFO:root:Using GPU


total:11, prediction_length:10
top1acc= 0.4 top1acc_farmost= 0.2727272727272727 top5acc= 0.48 top5acc_farmost= 0.2909090909090909
tau =  0.49364913122141263 rmse =  43.22325478608238
model: zerotrack
predicting model=zerotrack, plen=10
loading model...done!, ctx:gpu(0)
tss len=284, forecasts len=284
total:11, prediction_length:10
top1acc= 0.4090909090909091 top1acc_farmost= 0.2727272727272727 top5acc= 0.5381818181818182 top5acc_farmost= 0.38181818181818183
tau =  0.5296862710476785 rmse =  37.96041397044846


ValueError: could not convert string to float: 'indy500'

INFO:root:Using GPU


====event:Indy500, train_len=160, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:19
a short ts: carid=3，len=146
carno:4, totallen:200, nancount:0, test_reccnt:19
carno:6, totallen:200, nancount:0, test_reccnt:19
carno:7, totallen:193, nancount:7, test_reccnt:15
carno:9, totallen:200, nancount:0, test_reccnt:19
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:19
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:12
carno:15, totallen:200, nancount:0, test_reccnt:19
carno:17, totallen:199, nancount:1, test_reccnt:18
a short ts: carid=18，len=137
carno:19, totallen:199, nancount:1, test_reccnt:18
carno:20, totallen:200, nancount:0, test_reccnt:19
carno:21, totallen:199, nancount:1, test_reccnt:18
carno:22, totallen:200, nancount:0, test_reccnt:19
carno:23, totallen:200, nancount:0, test_reccnt:19
a short ts: carid=24，len=154
carno:25, totallen:200, nancount:0, test_reccnt:19
carno:26, totallen:198, nancount:2, te

INFO:root:Using GPU


total:19, prediction_length:2
top1acc= 0.631578947368421 top1acc_farmost= 0.5789473684210527 top5acc= 0.7947368421052632 top5acc_farmost= 0.7157894736842105
tau =  0.8172866707184919 rmse =  11.172209509280448
model: curtrack
predicting model=curtrack, plen=2
loading model...done!, ctx:gpu(0)


INFO:root:Using GPU


...

total:7, prediction_length:5
top1acc= 0.42857142857142855 top1acc_farmost= 0.42857142857142855 top5acc= 0.6114285714285714 top5acc_farmost= 0.5428571428571428
tau =  0.6278372663426373 rmse =  24.92719254658385
====event:Indy500, train_len=160, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:3
a short ts: carid=3，len=146
carno:4, totallen:200, nancount:0, test_reccnt:3
carno:6, totallen:200, nancount:0, test_reccnt:3
carno:7, totallen:193, nancount:7, test_reccnt:2
carno:9, totallen:200, nancount:0, test_reccnt:3
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:3
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:1
carno:15, totallen:200, nancount:0, test_reccnt:3
carno:17, totallen:199, nancount:1, test_reccnt:2
a short ts: carid=18，len=137
carno:19, totallen:199, nancount:1, test_reccnt:2
carno:20, totallen:200, nancount:0, test_reccnt:3
carno:21, totallen:199, nancount:1, test_reccnt:2
carno:22, totallen:20

INFO:root:Using GPU


tss len=67, forecasts len=67
total:3, prediction_length:10
top1acc= 0.26666666666666666 top1acc_farmost= 0.0 top5acc= 0.4666666666666667 top5acc_farmost= 0.4
tau =  0.38166375864355406 rmse =  38.27374074074074
model: curtrack
predicting model=curtrack, plen=10
loading model...done!, ctx:gpu(0)
tss len=67, forecasts len=67


INFO:root:Using GPU


total:3, prediction_length:10
top1acc= 0.1 top1acc_farmost= 0.0 top5acc= 0.36666666666666664 top5acc_farmost= 0.2
tau =  0.3222085251491902 rmse =  41.659333333333336
model: zerotrack
predicting model=zerotrack, plen=10
loading model...done!, ctx:gpu(0)
tss len=67, forecasts len=67
total:3, prediction_length:10
top1acc= 0.3 top1acc_farmost= 0.0 top5acc= 0.47333333333333333 top5acc_farmost= 0.4666666666666667
tau =  0.41607126077484136 rmse =  36.13562962962963


In [39]:
train_ratio=0.8
dfret.to_csv(f'laptime2rank-evaluate-indy500-{exp_id}-r{runs}-t{train_ratio}-result.csv')

In [35]:
half=[True, False]
#plens=[2,5,10,20,30]
plens=[2,5,10]
half=[False]
trainids = ["indy500-r0.2","indy500-r0.4","indy500"]
#trainids = ["r0.5","r0.6"]
#half=[True,False]
#plens=[2]
runs = 5

#exp_id='median-r0.8'
exp_id='mean-splitbyevent-predtrack'



allret = []
for runid in range(runs):
    exp_data = []
    exp_result = []
    
    for halfmode in half:
        for plen in plens:
            for trainid in trainids:
                print('='*30)
                pred_ret, test_ds, metric_ret = run_exp_predtrack(plen, halfmode, 
                                                        train_ratio=0.4,
                                                        trainid=trainid)

                #save 
                exp_data.append((pred_ret, test_ds))
                exp_result.extend(metric_ret)

    #save result
    result = pd.DataFrame(exp_result, columns = ['model' , 'prediction_length', 'halfmode',
                                       'trainid',
                                       'top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse'])
    
    #result['runid'] = [runid for x in range(len(result))]
    allret.append(result)


#final
rowcnt = len(allret[0])
metrics = np.empty((runs, rowcnt, 6))
for runid, ret in enumerate(allret):
    metrics[runid, :,:] = ret[['top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse']].values
    

#average
average = np.mean(metrics[:,:,:], axis=0)
result = allret[0][['model' , 'prediction_length', 'halfmode',
                                       'trainid',
                               'top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse']].values
result[:,4:] = average

dfret = pd.DataFrame(result, columns = ['model' , 'prediction_length', 'halfmode',
                                   'trainid',
                                   'top1acc','top1acc_farmost','top5acc',
                                   'top5acc_farmost','tau','rmse'])

dfret.to_csv(f'laptime2rank-evaluate-indy500-{exp_id}-r{runs}-result.csv')



INFO:root:Using GPU


====event:Indy500, train_len=80, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:59
carno:3, totallen:146, nancount:54, test_reccnt:32
carno:4, totallen:200, nancount:0, test_reccnt:59
carno:6, totallen:200, nancount:0, test_reccnt:59
carno:7, totallen:193, nancount:7, test_reccnt:55
carno:9, totallen:200, nancount:0, test_reccnt:59
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:59
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:52
carno:15, totallen:200, nancount:0, test_reccnt:59
carno:17, totallen:199, nancount:1, test_reccnt:58
carno:18, totallen:137, nancount:63, test_reccnt:27
carno:19, totallen:199, nancount:1, test_reccnt:58
carno:20, totallen:200, nancount:0, test_reccnt:59
carno:21, totallen:199, nancount:1, test_reccnt:58
carno:22, totallen:200, nancount:0, test_reccnt:59
carno:23, totallen:200, nancount:0, test_reccnt:59
carno:24, totallen:154, nancount:46, test_reccnt:36
carno:25, totallen:20

INFO:root:Using GPU


total:59, prediction_length:2
top1acc= 0.652542372881356 top1acc_farmost= 0.576271186440678 top5acc= 0.7983050847457627 top5acc_farmost= 0.7389830508474576
tau =  0.8272684733685166 rmse =  12.23668552367305
model: curtrack
predicting model=curtrack, plen=2
loading model...done!, ctx:gpu(0)
tss len=1568, forecasts len=1568


INFO:root:Using GPU


...

total:11, prediction_length:10
top1acc= 0.2545454545454545 top1acc_farmost= 0.18181818181818182 top5acc= 0.48363636363636364 top5acc_farmost= 0.32727272727272727
tau =  0.49449145100423875 rmse =  41.857777502191304
====event:Indy500, train_len=80, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:11
carno:3, totallen:146, nancount:54, test_reccnt:5
carno:4, totallen:200, nancount:0, test_reccnt:11
carno:6, totallen:200, nancount:0, test_reccnt:11
carno:7, totallen:193, nancount:7, test_reccnt:10
carno:9, totallen:200, nancount:0, test_reccnt:11
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:11
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:9
carno:15, totallen:200, nancount:0, test_reccnt:11
carno:17, totallen:199, nancount:1, test_reccnt:10
carno:18, totallen:137, nancount:63, test_reccnt:4
carno:19, totallen:199, nancount:1, test_reccnt:10
carno:20, totallen:200, nancount:0, test_reccnt:11
carno:21, tot

INFO:root:Using GPU


total:11, prediction_length:10
top1acc= 0.35454545454545455 top1acc_farmost= 0.18181818181818182 top5acc= 0.52 top5acc_farmost= 0.36363636363636365
tau =  0.5024937283765841 rmse =  41.515462634428154
model: curtrack
predicting model=curtrack, plen=10
loading model...done!, ctx:gpu(0)
tss len=284, forecasts len=284


INFO:root:Using GPU


total:11, prediction_length:10
top1acc= 0.33636363636363636 top1acc_farmost= 0.18181818181818182 top5acc= 0.5581818181818182 top5acc_farmost= 0.36363636363636365
tau =  0.525489138909811 rmse =  39.18131677262711
model: zerotrack
predicting model=zerotrack, plen=10
loading model...done!, ctx:gpu(0)
tss len=284, forecasts len=284
total:11, prediction_length:10
top1acc= 0.33636363636363636 top1acc_farmost= 0.18181818181818182 top5acc= 0.5781818181818181 top5acc_farmost= 0.4
tau =  0.535314610031243 rmse =  37.20564726588864


In [43]:
#plens=[2,5,10,20,30]
plens=[2,5,10]
half=[False]
#trainids = ["indy500-r0.2","indy500-r0.4","indy500"]
trainids = ["r0.5"]
#half=[True,False]
#plens=[2]
runs = 5
train_ratio=0.5 

#exp_id='median-r0.8'
exp_id='mean-splitbystage-predpit'

allret = []
for runid in range(runs):
    exp_data = []
    exp_result = []
    
    for halfmode in half:
        for plen in plens:
            for trainid in trainids:
                print('='*30)
                pred_ret, test_ds, metric_ret = run_exp_predpit(plen, halfmode, 
                                                        train_ratio=train_ratio,
                                                        trainid=trainid)

                #save 
                exp_data.append((pred_ret, test_ds))
                exp_result.extend(metric_ret)

    #save result
    result = pd.DataFrame(exp_result, columns = ['model' , 'prediction_length', 'halfmode',
                                       'trainid',
                                       'top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse'])
    
    #result['runid'] = [runid for x in range(len(result))]
    allret.append(result)


#final
rowcnt = len(allret[0])
metrics = np.empty((runs, rowcnt, 6))
for runid, ret in enumerate(allret):
    metrics[runid, :,:] = ret[['top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse']].values
    

#average
average = np.mean(metrics[:,:,:], axis=0)
result = allret[0][['model' , 'prediction_length', 'halfmode',
                                       'trainid',
                               'top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse']].values
result[:,4:] = average

dfret = pd.DataFrame(result, columns = ['model' , 'prediction_length', 'halfmode',
                                   'trainid',
                                   'top1acc','top1acc_farmost','top5acc',
                                   'top5acc_farmost','tau','rmse'])

dfret.to_csv(f'laptime2rank-evaluate-indy500-{exp_id}-r{runs}-t{train_ratio}-result.csv')

====event:Indy500, train_len=100, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:49
carno:3, totallen:146, nancount:54, test_reccnt:22
carno:4, totallen:200, nancount:0, test_reccnt:49
carno:6, totallen:200, nancount:0, test_reccnt:49
carno:7, totallen:193, nancount:7, test_reccnt:45
carno:9, totallen:200, nancount:0, test_reccnt:49
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:49
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:42
carno:15, totallen:200, nancount:0, test_reccnt:49
carno:17, totallen:199, nancount:1, test_reccnt:48
carno:18, totallen:137, nancount:63, test_reccnt:17
carno:19, totallen:199, nancount:1, test_reccnt:48
carno:20, totallen:200, nancount:0, test_reccnt:49
carno:21, totallen:199, nancount:1, test_reccnt:48
carno:22, totallen:200, nancount:0, test_reccnt:49
carno:23, totallen:200, nancount:0, test_reccnt:49
carno:24, totallen:154, nancount:46, test_reccnt:26
carno:25, totallen:2

INFO:root:Using GPU


model: oracle
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
tss len=1278, forecasts len=1278


INFO:root:Using GPU


total:49, prediction_length:2
top1acc= 0.6020408163265306 top1acc_farmost= 0.5510204081632653 top5acc= 0.7938775510204081 top5acc_farmost= 0.7306122448979592
tau =  0.8105315228319764 rmse =  12.42844865760785
model: curtrack
predicting model=curtrack, plen=2
loading model...done!, ctx:gpu(0)
tss len=1278, forecasts len=1278


...

total:19, prediction_length:5
top1acc= 0.43157894736842106 top1acc_farmost= 0.3157894736842105 top5acc= 0.5894736842105263 top5acc_farmost= 0.49473684210526314
tau =  0.629581903230089 rmse =  28.424905232044345
====event:Indy500, train_len=100, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:9
carno:3, totallen:146, nancount:54, test_reccnt:3
carno:4, totallen:200, nancount:0, test_reccnt:9
carno:6, totallen:200, nancount:0, test_reccnt:9
carno:7, totallen:193, nancount:7, test_reccnt:8
carno:9, totallen:200, nancount:0, test_reccnt:9
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:9
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:7
carno:15, totallen:200, nancount:0, test_reccnt:9
carno:17, totallen:199, nancount:1, test_reccnt:8
carno:18, totallen:137, nancount:63, test_reccnt:2
carno:19, totallen:199, nancount:1, test_reccnt:8
carno:20, totallen:200, nancount:0, test_reccnt:9
carno:21, totallen:199, na

INFO:root:Using GPU


total:9, prediction_length:10
top1acc= 0.28888888888888886 top1acc_farmost= 0.2222222222222222 top5acc= 0.4777777777777778 top5acc_farmost= 0.35555555555555557
tau =  0.45799834558209107 rmse =  41.89720146520147
model: curtrack
predicting model=curtrack, plen=10
loading model...done!, ctx:gpu(0)
tss len=226, forecasts len=226


INFO:root:Using GPU


total:9, prediction_length:10
top1acc= 0.34444444444444444 top1acc_farmost= 0.2222222222222222 top5acc= 0.5177777777777778 top5acc_farmost= 0.3333333333333333
tau =  0.5123908122347167 rmse =  37.15918830552163
model: zerotrack
predicting model=zerotrack, plen=10
loading model...done!, ctx:gpu(0)
tss len=226, forecasts len=226
total:9, prediction_length:10
top1acc= 0.2777777777777778 top1acc_farmost= 0.1111111111111111 top5acc= 0.48444444444444446 top5acc_farmost= 0.37777777777777777
tau =  0.5168992496457686 rmse =  34.522448424003976


In [44]:
half=[True, False]
#plens=[2,5,10,20,30]
plens=[2,5,10]
half=[False]
#trainids = ["indy500-r0.2","indy500-r0.4","indy500"]
trainids = ["r0.5"]
#half=[True,False]
#plens=[2]
runs = 5

#exp_id='median-r0.8'
exp_id='mean-splitbystage-predtrack'



allret = []
for runid in range(runs):
    exp_data = []
    exp_result = []
    
    for halfmode in half:
        for plen in plens:
            for trainid in trainids:
                print('='*30)
                pred_ret, test_ds, metric_ret = run_exp_predtrack(plen, halfmode, 
                                                        train_ratio=train_ratio,
                                                        trainid=trainid)

                #save 
                exp_data.append((pred_ret, test_ds))
                exp_result.extend(metric_ret)

    #save result
    result = pd.DataFrame(exp_result, columns = ['model' , 'prediction_length', 'halfmode',
                                       'trainid',
                                       'top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse'])
    
    #result['runid'] = [runid for x in range(len(result))]
    allret.append(result)


#final
rowcnt = len(allret[0])
metrics = np.empty((runs, rowcnt, 6))
for runid, ret in enumerate(allret):
    metrics[runid, :,:] = ret[['top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse']].values
    

#average
average = np.mean(metrics[:,:,:], axis=0)
result = allret[0][['model' , 'prediction_length', 'halfmode',
                                       'trainid',
                               'top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse']].values
result[:,4:] = average

dfret = pd.DataFrame(result, columns = ['model' , 'prediction_length', 'halfmode',
                                   'trainid',
                                   'top1acc','top1acc_farmost','top5acc',
                                   'top5acc_farmost','tau','rmse'])

dfret.to_csv(f'laptime2rank-evaluate-indy500-{exp_id}-r{runs}-t{train_ratio}-result.csv')

INFO:root:Using GPU


====event:Indy500, train_len=100, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:49
carno:3, totallen:146, nancount:54, test_reccnt:22
carno:4, totallen:200, nancount:0, test_reccnt:49
carno:6, totallen:200, nancount:0, test_reccnt:49
carno:7, totallen:193, nancount:7, test_reccnt:45
carno:9, totallen:200, nancount:0, test_reccnt:49
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:49
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:42
carno:15, totallen:200, nancount:0, test_reccnt:49
carno:17, totallen:199, nancount:1, test_reccnt:48
carno:18, totallen:137, nancount:63, test_reccnt:17
carno:19, totallen:199, nancount:1, test_reccnt:48
carno:20, totallen:200, nancount:0, test_reccnt:49
carno:21, totallen:199, nancount:1, test_reccnt:48
carno:22, totallen:200, nancount:0, test_reccnt:49
carno:23, totallen:200, nancount:0, test_reccnt:49
carno:24, totallen:154, nancount:46, test_reccnt:26
carno:25, totallen:2

INFO:root:Using GPU


total:49, prediction_length:2
top1acc= 0.6020408163265306 top1acc_farmost= 0.5510204081632653 top5acc= 0.8061224489795918 top5acc_farmost= 0.7551020408163265
tau =  0.8283667707906474 rmse =  10.829043207520856
model: curtrack
predicting model=curtrack, plen=2
loading model...done!, ctx:gpu(0)
tss len=1278, forecasts len=1278


INFO:root:Using GPU


...

total:19, prediction_length:5
top1acc= 0.4421052631578947 top1acc_farmost= 0.2631578947368421 top5acc= 0.5705263157894737 top5acc_farmost= 0.4842105263157895
tau =  0.6235236122028274 rmse =  30.08152451934177
====event:Indy500, train_len=100, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:9
carno:3, totallen:146, nancount:54, test_reccnt:3
carno:4, totallen:200, nancount:0, test_reccnt:9
carno:6, totallen:200, nancount:0, test_reccnt:9
carno:7, totallen:193, nancount:7, test_reccnt:8
carno:9, totallen:200, nancount:0, test_reccnt:9
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:9
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:7
carno:15, totallen:200, nancount:0, test_reccnt:9
carno:17, totallen:199, nancount:1, test_reccnt:8
carno:18, totallen:137, nancount:63, test_reccnt:2
carno:19, totallen:199, nancount:1, test_reccnt:8
carno:20, totallen:200, nancount:0, test_reccnt:9
carno:21, totallen:199, nanc

INFO:root:Using GPU


total:9, prediction_length:10
top1acc= 0.28888888888888886 top1acc_farmost= 0.2222222222222222 top5acc= 0.5377777777777778 top5acc_farmost= 0.35555555555555557
tau =  0.5093979387551708 rmse =  35.403856645412205
model: curtrack
predicting model=curtrack, plen=10
loading model...done!, ctx:gpu(0)
tss len=226, forecasts len=226


INFO:root:Using GPU


total:9, prediction_length:10
top1acc= 0.36666666666666664 top1acc_farmost= 0.2222222222222222 top5acc= 0.56 top5acc_farmost= 0.4222222222222222
tau =  0.5401926667293533 rmse =  31.764083073305294
model: zerotrack
predicting model=zerotrack, plen=10
loading model...done!, ctx:gpu(0)
tss len=226, forecasts len=226
total:9, prediction_length:10
top1acc= 0.2777777777777778 top1acc_farmost= 0.1111111111111111 top5acc= 0.48 top5acc_farmost= 0.3333333333333333
tau =  0.5088503261381352 rmse =  35.690177361732914


In [45]:
#
# baselines
#

half=[True, False]
#plens=[2,5,10,20,30]
plens=[2,5,10]
half=[False]
#trainids = ["indy500-r0.2","indy500-r0.4","indy500"]
trainids = ["r0.5"]
#half=[True,False]
#plens=[2]
runs = 5

#exp_id='median-r0.8'
exp_id='mean-splitbystage-baseline'



allret = []
for runid in range(runs):
    exp_data = []
    exp_result = []
    
    for halfmode in half:
        for plen in plens:
            for trainid in trainids:
                print('='*30)
                pred_ret, test_ds, metric_ret = run_exp(plen, halfmode, 
                                                        train_ratio=train_ratio,
                                                        trainid=trainid)

                #save 
                exp_data.append((pred_ret, test_ds))
                exp_result.extend(metric_ret)

    #save result
    result = pd.DataFrame(exp_result, columns = ['model' , 'prediction_length', 'halfmode',
                                       'trainid',
                                       'top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse'])
    
    #result['runid'] = [runid for x in range(len(result))]
    allret.append(result)


#final
rowcnt = len(allret[0])
metrics = np.empty((runs, rowcnt, 6))
for runid, ret in enumerate(allret):
    metrics[runid, :,:] = ret[['top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse']].values
    

#average
average = np.mean(metrics[:,:,:], axis=0)
result = allret[0][['model' , 'prediction_length', 'halfmode',
                                       'trainid',
                               'top1acc','top1acc_farmost','top5acc',
                                       'top5acc_farmost','tau','rmse']].values
result[:,4:] = average

dfret = pd.DataFrame(result, columns = ['model' , 'prediction_length', 'halfmode',
                                   'trainid',
                                   'top1acc','top1acc_farmost','top5acc',
                                   'top5acc_farmost','tau','rmse'])

dfret.to_csv(f'laptime2rank-evaluate-indy500-{exp_id}-r{runs}-t{train_ratio}-result.csv')

INFO:root:Using GPU


====event:Indy500, train_len=100, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:49
carno:3, totallen:146, nancount:54, test_reccnt:22
carno:4, totallen:200, nancount:0, test_reccnt:49
carno:6, totallen:200, nancount:0, test_reccnt:49
carno:7, totallen:193, nancount:7, test_reccnt:45
carno:9, totallen:200, nancount:0, test_reccnt:49
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:49
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:42
carno:15, totallen:200, nancount:0, test_reccnt:49
carno:17, totallen:199, nancount:1, test_reccnt:48
carno:18, totallen:137, nancount:63, test_reccnt:17
carno:19, totallen:199, nancount:1, test_reccnt:48
carno:20, totallen:200, nancount:0, test_reccnt:49
carno:21, totallen:199, nancount:1, test_reccnt:48
carno:22, totallen:200, nancount:0, test_reccnt:49
carno:23, totallen:200, nancount:0, test_reccnt:49
carno:24, totallen:154, nancount:46, test_reccnt:26
carno:25, totallen:2

INFO:root:Using GPU


total:49, prediction_length:2
top1acc= 0.7448979591836735 top1acc_farmost= 0.6938775510204082 top5acc= 0.8571428571428571 top5acc_farmost= 0.8204081632653061
tau =  0.8732123728695564 rmse =  6.169249661319697
model: deepAR
predicting model=deepAR, plen=2
loading model...done!, ctx:gpu(0)
tss len=1278, forecasts len=1278
total:49, prediction_length:2
top1acc= 0.5 top1acc_farmost= 0.42857142857142855 top5acc= 0.7387755102040816 top5acc_farmost= 0.6571428571428571
tau =  0.7614259279269916 rmse =  15.335019059083251
model: naive
predicting model=naive, plen=2


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/NaivePredictor.py:60: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  start_date=start + target_len,


...

tss len=226, forecasts len=226


INFO:root:Using GPU


total:9, prediction_length:10
top1acc= 0.3 top1acc_farmost= 0.2222222222222222 top5acc= 0.4444444444444444 top5acc_farmost= 0.28888888888888886
tau =  0.4102774175725867 rmse =  46.28773694207028
====event:Indy500, train_len=100, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:9
carno:3, totallen:146, nancount:54, test_reccnt:3
carno:4, totallen:200, nancount:0, test_reccnt:9
carno:6, totallen:200, nancount:0, test_reccnt:9
carno:7, totallen:193, nancount:7, test_reccnt:8
carno:9, totallen:200, nancount:0, test_reccnt:9
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:9
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:7
carno:15, totallen:200, nancount:0, test_reccnt:9
carno:17, totallen:199, nancount:1, test_reccnt:8
carno:18, totallen:137, nancount:63, test_reccnt:2
carno:19, totallen:199, nancount:1, test_reccnt:8
carno:20, totallen:200, nancount:0, test_reccnt:9
carno:21, totallen:199, nancount:1, test_r

INFO:root:Using GPU


total:9, prediction_length:10
top1acc= 0.3 top1acc_farmost= 0.1111111111111111 top5acc= 0.5244444444444445 top5acc_farmost= 0.4
tau =  0.5055291147897285 rmse =  38.16860548093881
====event:Indy500, train_len=100, max_len=200, min_len=200
carno:1, totallen:200, nancount:0, test_reccnt:9
carno:3, totallen:146, nancount:54, test_reccnt:3
carno:4, totallen:200, nancount:0, test_reccnt:9
carno:6, totallen:200, nancount:0, test_reccnt:9
carno:7, totallen:193, nancount:7, test_reccnt:8
carno:9, totallen:200, nancount:0, test_reccnt:9
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:9
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:7
carno:15, totallen:200, nancount:0, test_reccnt:9
carno:17, totallen:199, nancount:1, test_reccnt:8
carno:18, totallen:137, nancount:63, test_reccnt:2
carno:19, totallen:199, nancount:1, test_reccnt:8
carno:20, totallen:200, nancount:0, test_reccnt:9
carno:21, totallen:199, nancount:1, test_reccnt:8
carno:22

### test

In [16]:
result[result["prediction_length"]==2]

,model,prediction_length,halfmode,trainid,top1acc,top1acc_farmost,top5acc,top5acc_farmost,tau,rmse
0,oracle,2,True,r0.5,0.697368,0.631579,0.810526,0.784211,0.856538,6.495403
1,deepAR,2,True,r0.5,0.381579,0.315789,0.681579,0.615789,0.713712,16.446900
2,naive,2,True,r0.5,0.618421,0.578947,0.813158,0.768421,0.795573,14.307786
3,curtrack,2,True,r0.5,0.473684,0.368421,0.789474,0.742105,0.812070,10.850776
4,zerotrack,2,True,r0.5,0.473684,0.447368,0.802632,0.752632,0.808004,10.236402
5,oracle,2,True,r0.6,0.789474,0.789474,0.868421,0.831579,0.887596,5.271319
6,deepAR,2,True,r0.6,0.473684,0.368421,0.739474,0.678947,0.782108,13.329318
7,naive,2,True,r0.6,0.618421,0.578947,0.813158,0.768421,0.795573,14.307786
8,curtrack,2,True,r0.6,0.605263,0.552632,0.807895,0.757895,0.837663,9.677535
9,zerotrack,2,True,r0.6,0.605263,0.526316,0.797368,0.742105,0.826675,9.016000


In [ ]:
tss, forecasts = exp_data[0][0]['naive']

In [ ]:
forecasts[0].samples.shape

In [ ]:
[x[3][:,:].shape for x in rank_ret]

In [ ]:
rank_ret[x][2][:,:]

In [ ]:
rank_ret[1][3][:,:]

In [ ]:
pred_ret.keys()

In [ ]:
forecasts[0].samples.shape

In [ ]:
start_offset

In [ ]:
result[result['prediction_length']==10]

In [27]:
_track_pred

{200: array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
 190: array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0]),
 180: array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0])}

In [26]:
_track_true

{200: array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 190: array([1., 1., 1., 1., 1., 1., 0., 0., 0., 0.]),
 180: array([1., 1., 1., 1., 1., 1., 0., 0., 0., 0.])}

In [15]:
pitpred = np.array([0 for x in range(10)])
pitpred.shape

(10,)

In [16]:
pitpred[0] 

0